<a target="_blank" href="https://colab.research.google.com/github/korakoe/Wuerstchen/blob/main/Train_w%C3%BCrstchen.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<p align="center"><img src="https://raw.githubusercontent.com/ShoukanLabs/.github/main/profile/ShoukanLab-circle.png" width="256"></p>

<p align="center", style="font-size: 2rem; font-weight: bold; color: #ff593e;">🧪 ShoukanLabs ー召喚Labsー</p>

<p align="center">Shoukan 召喚【しょうかん】 ー can be translated into either Summon or Summoning.</p>

<p align="center" style="font-size: 1rem; font-weight: bold; color: #ff593e;">Join our Discord at: https://discord.gg/5bq9HqVhsJ</p>

---
## 🔎 What is this notebook?

This is a small training notebook for wurstchen, as the official training script requires ddp, and doesnt have argparse, as such ive taken the time to add these to the script, you can find my github [here](https://github.com/korakoe/Wuerstchen)

---

#### 📚 In the mean time... check out our other projects

- **Future Projects**
  - Unnamed TTS project
  - OpenNiji-V3 *(placeholder name)*

- **Current Projects**
  - [OpenNiji-Dataset](https://huggingface.co/datasets/ShoukanLabs/OpenNiji-Dataset) - The dataset that will be used for OpenNiji-V3
    - [Dataset preview](https://huggingface.co/spaces/ShoukanLabs/OpenNiji-Dataset-Viewer) - See what images the dataset contains

- **Previous Projects**
  - [OpenNiji](https://huggingface.co/ShoukanLabs/OpenNiji) - A finetune aimed at replicating Nijijourney on Stable Diffusion.
  - [OpenNiji-V2](https://huggingface.co/ShoukanLabs/OpenNiji-V2) - A second finetune made to replicate the Nijijourney style more accurately.

In [17]:
#@title Create Ennvrionment

!rm -r Wuerstchen
!git clone https://github.com/korakoe/Wuerstchen.git
!pip install -r Wuerstchen/requirements.txt
!pip install pip install git+https://github.com/pabloppp/pytorch-tools@master
!pip install wandb transformers webdataset datasets warmup_scheduler

import wandb
print("Logging into W&B...")
wandb.login()

from google.colab import drive
print("Mounting Drive...")
drive.mount('/content/drive')

Cloning into 'Wuerstchen'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 183 (delta 51), reused 53 (delta 25), pack-reused 92
Receiving objects: 100% (183/183), 44.85 MiB | 22.82 MiB/s, done.
Resolving deltas: 100% (98/98), done.
  Cloning https://github.com/pabloppp/pytorch-tools (to revision master) to /tmp/pip-install-omc7z6sr/pytorch-tools_cb251e2dbbed480a90148f0974ca4bf2
  Running command git clone --filter=blob:none --quiet https://github.com/pabloppp/pytorch-tools /tmp/pip-install-omc7z6sr/pytorch-tools_cb251e2dbbed480a90148f0974ca4bf2
  Resolved https://github.com/pabloppp/pytorch-tools to commit c99c4c1779b66429044cc686fc94c659bc5f5dee
  Preparing metadata (setup.py) ... done
Discarding git+https://github.com/pabloppp/pytorch-tools@master: Requested torchtools from git+https://github.com/pabloppp/pytorch-tools@master (from -r Wuerstchen/requirements.txt (line 4)) has incons

In [2]:
#@title Download Required Models For Training
!wget https://huggingface.co/dome272/wuerstchen/resolve/main/vqgan_f4_v1_500k.pt -O "vq.pt"
!wget https://huggingface.co/dome272/wuerstchen/resolve/main/model_v2_stage_b.pt -O "stage_b.bin"
!wget https://huggingface.co/dome272/wuerstchen/resolve/main/model_v2_stage_c.pt -O "stage_c.bin"

--2023-09-30 07:35:02--  https://huggingface.co/dome272/wuerstchen/resolve/main/vqgan_f4_v1_500k.pt
Resolving huggingface.co (huggingface.co)... 3.163.180.7, 3.163.180.117, 3.163.180.30, ...
Connecting to huggingface.co (huggingface.co)|3.163.180.7|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b9/6e/b96ebc9f17a640b8e89f419a072a1ef4e6dadbcdf69a668bf2518fc87cd52c63/b4a81dd8733268a68d08844e33b917d4dbe27d5c5382dcbde522df3af16f343e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27vqgan_f4_v1_500k.pt%3B+filename%3D%22vqgan_f4_v1_500k.pt%22%3B&Expires=1696318503&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NjMxODUwM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iOS82ZS9iOTZlYmM5ZjE3YTY0MGI4ZTg5ZjQxOWEwNzJhMWVmNGU2ZGFkYmNkZjY5YTY2OGJmMjUxOGZjODdjZDUyYzYzL2I0YTgxZGQ4NzMzMjY4YTY4ZDA4ODQ0ZTMzYjkxN2Q0ZGJlMjdkNWM1MzgyZGNiZGU1MjJkZjNhZjE2ZjM0M2U%

In [18]:
# @title Start Training stage B
trained_vq_path = "./vq.pt" # @param {type:"string"}
steps = 1500000 # @param {type:"integer"}
warmup_steps = 10000 # @param {type:"integer"}
ema_start = 5000 # @param {type:"integer"}
ema_every = 100 # @param {type:"integer"}
ema_beta = 0.9 # @param {type:"number"}
batch_size = 8 # @param {type:"integer"}
grad_accum_steps = 1 # @param {type:"integer"}
print_every = 5 # @param {type:"integer"}
extra_ckpt_every = 1000 # @param {type:"integer"}
learning_rate = 1e-4 # @param {type:"number"}
HF_dataset_name = "ShoukanLabs/OpenNiji-0_32237" # @param {type:"string"}
cache_path = "./drive/MyDrive/HF_Cache" # @param {type:"string"}
text_column = "prompt" # @param {type:"string"}
image_column = "image" # @param {type:"string"}
run_name = "OpenNiji-V3" # @param {type:"string"}
output_path = "./drive/MyDrive/OpenNiji-v3" # @param {type:"string"}
# @markdown ^^ output_path is for eval images, not for saving the model ^^
load_from_pretrained = True # @param {type:"boolean"}
load_path = "./stage_b.bin" # @param {type:"string"}
save_path = "./drive/MyDrive/OpenNiji-v3" # @param {type:"string"}

if load_from_pretrained:
  !python Wuerstchen/train_stage_B.py \
  --vq_model_path $trained_vq_path \
  --updates $steps \
  --warmup_updates $warmup_steps \
  --ema_start $ema_start \
  --ema_every $ema_every \
  --ema_beta $ema_beta \
  --batch_size $batch_size \
  --grad_accum_steps $grad_accum_steps \
  --print_every $print_every \
  --extra_ckpt_every $extra_ckpt_every \
  --lr $learning_rate \
  --hf_dataset_name $HF_dataset_name \
  --text_column $text_column \
  --image_column $image_column \
  --run_name $run_name \
  --output_path $output_path \
  --load \
  --load_checkpoint_path $load_path \
  --cache_path $cache_path \
  --save_checkpoint_path $save_path
else:
  !python Wuerstchen/train_stage_B.py \
  --vq_model_path $trained_vq_path \
  --updates $steps \
  --warmup_updates $warmup_steps \
  --ema_start $ema_start \
  --ema_every $ema_every \
  --ema_beta $ema_beta \
  --batch_size $batch_size \
  --grad_accum_steps $grad_accum_steps \
  --print_every $print_every \
  --extra_ckpt_every $extra_ckpt_every \
  --lr $learning_rate \
  --hf_dataset_name $HF_dataset_name \
  --cache_path $cache_path \
  --text_column $text_column \
  --image_column $image_column \
  --run_name $run_name \
  --output_path $output_path \
  --save_checkpoint_path $save_path


2023-09-30 10:34:18.927292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py:333: FutureWarning: The util is_torch_bf16_available is deprecated, please use is_torch_bf16_gpu_available or is_torch_bf16_cpu_available instead according to whether it's used with cpu or gpu
  warnings.warn(
Resolving data files: 100% 108/108 [00:06<00:00, 15.55it/s]
Traceback (most recent call last):
  File "/content/Wuerstchen/train_stage_B.py", line 472, in <module>
    train(0)
  File "/content/Wuerstchen/train_stage_B.py", line 238, in train
    dataset = load_dataset(
  File "/usr/local/lib/python3.10/dist-packages/datasets/load.py", line 2166, in load_dataset
    ds = builder_instance.as_dataset(split=split, verification_mode=verification_mode, in_memory=keep_in_memory)
  File "/usr/local/lib/python3.10/dist-packages/datasets/builder.py", line 1190, in as_dataset
    datasets = map

In [ ]:
# @title Start Training stage C
trained_stage_b_path = "" # @param {type:"string"}
trained_vq_path = "./vq.pt" # @param {type:"string"}
steps = 1500000 # @param {type:"integer"}
warmup_steps = 10000 # @param {type:"integer"}
ema_start = 5000 # @param {type:"integer"}
ema_every = 100 # @param {type:"integer"}
ema_beta = 0.9 # @param {type:"number"}
batch_size = 8 # @param {type:"integer"}
grad_accum_steps = 1 # @param {type:"integer"}
print_every = 5 # @param {type:"integer"}
extra_ckpt_every = 1000 # @param {type:"integer"}
learning_rate = 1e-4 # @param {type:"number"}
HF_dataset_name = "ShoukanLabs/OpenNiji-0_32237" # @param {type:"string"}
cache_path = "./drive/MyDrive/HF_Cache" # @param {type:"string"}
text_column = "prompt" # @param {type:"string"}
image_column = "image" # @param {type:"string"}
run_name = "OpenNiji-V3" # @param {type:"string"}
output_path = "./drive/MyDrive/OpenNiji-v3" # @param {type:"string"}
# @markdown ^^ output_path is for eval images, not for saving the model ^^
load_from_pretrained = True # @param {type:"boolean"}
load_path = "./stage_c.bin" # @param {type:"string"}
save_path = "./drive/MyDrive/OpenNiji-v3" # @param {type:"string"}

if load_from_pretrained:
  !python Wuerstchen/train_stage_B.py \
  --stage_b_path $trained_stage_b_path \
  --vq_model_path $trained_vq_path \
  --updates $steps \
  --warmup_updates $warmup_steps \
  --ema_start $ema_start \
  --ema_every $ema_every \
  --ema_beta $ema_beta \
  --batch_size $batch_size \
  --grad_accum_steps $grad_accum_steps \
  --print_every $print_every \
  --extra_ckpt_every $extra_ckpt_every \
  --lr $learning_rate \
  --hf_dataset_name $HF_dataset_name \
  --text_column $text_column \
  --image_column $image_column \
  --run_name $run_name \
  --output_path $output_path \
  --cache_path $cache_path \
  --load \
  --load_checkpoint_path $load_path \
  --save_checkpoint_path $save_path
else:
  !python Wuerstchen/train_stage_B.py \
  --stage_b_path $trained_stage_b_path \
  --vq_model_path $trained_vq_path \
  --updates $steps \
  --warmup_updates $warmup_steps \
  --ema_start $ema_start \
  --ema_every $ema_every \
  --ema_beta $ema_beta \
  --batch_size $batch_size \
  --grad_accum_steps $grad_accum_steps \
  --print_every $print_every \
  --extra_ckpt_every $extra_ckpt_every \
  --lr $learning_rate \
  --hf_dataset_name $HF_dataset_name \
  --cache_path $cache_path \
  --text_column $text_column \
  --image_column $image_column \
  --run_name $run_name \
  --output_path $output_path \
  --save_checkpoint_path $save_path


In [ ]:
# @title Test Finetuned Model
stage_c_path = "" # @param {type:"string"}
batch_size = 0 # @param {type:"integer"}
prompt = "A captivating artwork of a mysterious stone golem" # @param {type:"string"}
negative_prompt = "" # @param {type:"string"}

checkpoint_stage_a = "./vq.pt"
checkpoint_stage_b = trained_stage_b_path
checkpoint_stage_c = stage_c_path

effnet_preprocess = torchvision.transforms.Compose([
    torchvision.transforms.Resize(768, interpolation=torchvision.transforms.InterpolationMode.BILINEAR, antialias=True),
    torchvision.transforms.CenterCrop(768),
    torchvision.transforms.Normalize(
        mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
    )
])

def decode(img_seq):
    return vqmodel.decode(img_seq)

def embed_clip(clip_tokenizer, clip_model, caption, negative_caption="", batch_size=4, device="cuda"):
    clip_tokens = clip_tokenizer([caption] * batch_size, truncation=True, padding="max_length", max_length=clip_tokenizer.model_max_length, return_tensors="pt").to(device)
    clip_text_embeddings = clip_model(**clip_tokens).last_hidden_state

    clip_tokens_uncond = clip_tokenizer([negative_caption] * batch_size, truncation=True, padding="max_length", max_length=clip_tokenizer.model_max_length, return_tensors="pt").to(device)
    clip_text_embeddings_uncond = clip_model(**clip_tokens_uncond).last_hidden_state
    return clip_text_embeddings, clip_text_embeddings_uncond

vqmodel = VQModel().to(device)
vqmodel.load_state_dict(torch.load(checkpoint_stage_a, map_location=device)["state_dict"])
vqmodel.eval().requires_grad_(False)

clip_model = CLIPTextModel.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k").to(device).eval().requires_grad_(False)
clip_tokenizer = AutoTokenizer.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k")

clip_model_b = CLIPTextModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K").eval().requires_grad_(False).to(device)
clip_tokenizer_b = AutoTokenizer.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

diffuzz = Diffuzz(device=device)

pretrained_checkpoint = torch.load(checkpoint_stage_b, map_location=device)

effnet = EfficientNetEncoder().to(device)
effnet.load_state_dict(pretrained_checkpoint['effnet_state_dict'])
effnet.eval().requires_grad_(False)

# - LDM Model as generator -
generator = DiffNeXt()
generator.load_state_dict(pretrained_checkpoint['state_dict'])
generator.eval().requires_grad_(False).to(device)

del pretrained_checkpoint

checkpoint = torch.load(checkpoint_stage_c, map_location=device)
model = Prior(c_in=16, c=1536, c_cond=1280, c_r=64, depth=32, nhead=24).to(device)
model.load_state_dict(checkpoint['ema_state_dict'])
model.eval().requires_grad_(False)
del checkpoint

torch.cuda.empty_cache()

model = torch.compile(model, mode="reduce-overhead", fullgraph=True)
generator = torch.compile(generator, mode="reduce-overhead", fullgraph=True)

caption = prompt
negative_caption = negative_prompt
prior_inference_steps = {2/3: 20, 0.0: 10}
prior_cfg = 4
prior_sampler = "ddpm"

generator_steps = 24
generator_cfg = None
generator_sampler = "ddpm"

height = 1024
width = 1024

clip_text_embeddings, clip_text_embeddings_uncond = embed_clip(clip_tokenizer, clip_model, caption, negative_caption, batch_size, device)

latent_height = 128 * (height // 128) // (1024 // 24)
latent_width = 128 * (width // 128) // (1024 // 24)
prior_features_shape = (batch_size, 16, latent_height, latent_width)
effnet_embeddings_uncond = torch.zeros(effnet_features_shape).to(device)
generator_latent_shape = (batch_size, 4, int(latent_height * (256 / 24)), int(latent_width * (256 / 24)))
# torch.manual_seed(42)
with torch.cuda.amp.autocast(dtype=torch.bfloat16), torch.no_grad():
    s = time.time()
    t_start = 1.0
    sampled = None
    for t_end, steps in prior_inference_steps.items():
        sampled = diffuzz.sample(model, {'c': clip_text_embeddings}, x_init=sampled, unconditional_inputs={"c": clip_text_embeddings_uncond}, shape=prior_features_shape,
                            timesteps=steps, cfg=prior_cfg, sampler=prior_sampler,
                            t_start=t_start, t_end=t_end)[-1]
        t_start = t_end
    sampled = sampled.mul(42).sub(1)

    print(f"Prior Sampling: {time.time() - s}")

    clip_text_embeddings, clip_text_embeddings_uncond = embed_clip(clip_tokenizer_b, clip_model_b, caption, negative_caption, batch_size, device)

    s = time.time()
    sampled_images_original = diffuzz.sample(generator, {'effnet': sampled, 'clip': clip_text_embeddings},
                            generator_latent_shape, t_start=1.0, t_end=0.00,
                            timesteps=generator_steps, cfg=generator_cfg, sampler=generator_sampler,
                            unconditional_inputs = {
                            'effnet': effnet_embeddings_uncond, 'clip': clip_text_embeddings_uncond,
                        })[-1]
    print(f"Generator Sampling: {time.time() - s}")

s = time.time()
sampled = decode(sampled_images_original)
print(f"Decoder Generation: {time.time() - s}")
print(f"Prior => CFG: {prior_cfg}, Steps: {sum(prior_inference_steps.values())}, Sampler: {prior_sampler}")
print(f"Generator => CFG: {generator_cfg}, Steps: {generator_steps}, Sampler: {generator_sampler}")
print(f"Images Shape: {sampled.shape}")
print(caption)
showimages(sampled)